# P3 Concevez une application au service de la santé publique 

L'agence <span style='color:blue'> <a href = "http://www.santepubliquefrance.fr/">"*Santé publique France*" </a></span> a lancé un appel à projets pour trouver des idées innovantes d’applications en lien avec l'alimentation. <br>
Vous souhaitez y participer et proposer une idée d’application.

Le jeu de données <font color='orange'>Open Food Facts</font> est disponible sur le <a href = "https://world.openfoodfacts.org/">site officiel</a> (ou disponible à <a href ="https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip">ce lien</a> en téléchargement).
<br>
Les variables sont définies à <a href = "https://world.openfoodfacts.org/data/data-fields.txt">cette adresse</a>

Les champs sont séparés en quatre sections :

- Les informations générales sur la fiche du produit : nom, date de modification, etc.
- Un ensemble de tags : catégorie du produit, localisation, origine, etc.
- Les ingrédients composant les produits et leurs additifs éventuels.
- Des informations nutritionnelles : quantité en grammes d’un nutriment pour 100 grammes du produit.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import os

import missingno as msno

In [2]:
categories = pd.read_csv("fr.openfoodfacts.org.products.csv",
                         sep='\t',
                         usecols = ["main_category","main_category_fr","categories","categories","categories_fr","categories_tags"])

In [3]:
categories.head()

,categories,categories_tags,categories_fr,main_category,main_category_fr
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [4]:
categories = categories.dropna(how='all')

Nombre de ligne et de colonnes

In [5]:
nrows,ncols = categories.shape

In [6]:
nrows,ncols

(84412, 5)

In [7]:
categories.describe()

,categories,categories_tags,categories_fr,main_category,main_category_fr
count,84410,84389,84411,84366,84366
unique,36982,21142,21152,3543,3543
top,"Snacks sucrés,Biscuits et gâteaux,Biscuits","en:sugary-snacks,en:biscuits-and-cakes,en:bisc...","Snacks sucrés,Biscuits et gâteaux,Biscuits",en:beverages,Boissons
freq,301,802,802,6054,6054


In [8]:
categories.main_category_fr.str.replace('(?<=\\:).+','',regex=True).value_counts().head(20)

Boissons                                   6054
Epicerie                                   2902
Chocolats                                  2789
Aliments et boissons à base de végétaux    2745
Conserves                                  2519
Biscuits                                   2097
Plats préparés                             2079
Snacks sucrés                              1946
Surgelés                                   1889
Petit-déjeuners                            1793
Fromages                                   1785
Pains                                      1771
en:                                        1686
Desserts                                   1684
Pâtes alimentaires                         1529
Plats à base de viande                     1319
Gâteaux                                    1266
Chips et frites                            1222
Produits à tartiner salés                  1181
Bonbons                                    1113
Name: main_category_fr, dtype: int64

In [9]:
corr_main_categories_esdeptitpl = pd.read_excel('main_cateptplitesdeen.xlsx')
#corr_main_categories_esdeptitpl = corr_main_categories_esdeptitpl[corr_main_categories_esdeptitpl.traduit>0][["main_cate","translate english"]]

In [10]:
corr_main_categories_esdeptitpl.head()

,main_cate,langue origine,traduit
0,pt:Salgadinho-de-batata,Salgadinho de batata,En-cas à base de pommes de terre
1,pt:Cereais,Cereais,Céréales
2,pt:Molhos,Molhos,Sauces
3,pt:Refeicoes,Refeicoes,Repas
4,pt:Salgados,Salgados,Snacks


In [11]:
categories = categories.merge(right=corr_main_categories_esdeptitpl,
                 how='left',
                 left_on='main_category_fr',
                 right_on='main_cate')[['main_category','main_category_fr','traduit']]\
.drop_duplicates()

</br>

In [12]:
categories.describe()

,main_category,main_category_fr,traduit
count,3543,3543,1860
unique,3543,3543,1660
top,fr:filet-de-boeuf,Filet-de-boeuf,Produits laitiers
freq,1,1,9


In [13]:
### Correction lower - et ^[a-Z]:

In [14]:
categories.main_category_fr.str.lower().str.replace('-',' ',regex=True).str.replace('^[a-z]{2}\\:','',regex=True).sample(10)

30733          sauce pour risotto
3536                  indian food
42828                    asperges
946             grape juice drink
78969                      dulces
3914             lingonberry jams
41114    decorations alimentaires
1614              hot dog mustard
80584               tuiles salees
75521          gelatina per dolci
Name: main_category_fr, dtype: object

In [15]:
categories.traduit.sample(10)

10062                           Miel australien pur
30121                                           NaN
3519                                            NaN
71954                          Lait UHT homogénéisé
8143                          Epis de maïs congelés
46771                                           NaN
1140                                 Moutarde jaune
81584    Pâte à tartiner à base de graisse végétale
2328                              Sirop pour crêpes
42971                                           NaN
Name: traduit, dtype: object

In [16]:
categories['traduit'].str.lower()\
.str.replace('[éèëê]','e',regex=True)\
.str.replace('[àäâ]','a',regex=True)\
.str.replace('[ïîì]','i',regex=True)\
.str.replace('[ôöù]','o',regex=True)\
.str.replace('[ûüù]','u',regex=True)\
.str.replace('[œ]','oe',regex=True)\
.str.replace('[a-z ]','',regex=True).value_counts()

                                     1743
'                                      78
-                                      22
''                                      4
пряники                                 1
ı                                       1
-'                                      1
шоколадссолёнымминдалёмикарамелью       1
ç                                       1
светлоепиво                             1
биойогурты                              1
кетчуп                                  1
うまい棒                                    1
12                                      1
1                                       1
15                                      1
'ç                                      1
Name: traduit, dtype: int64

In [17]:
categories['main_category_fr_cor'] = [ y if y is not np.nan else x for x,y in zip(categories.main_category_fr,categories.traduit)]


categories['main_category_fr_cor'] = categories['main_category_fr_cor'].str.lower()\
.str.replace('-',' ',regex=True)\
.str.replace('[éèëê]','e',regex=True)\
.str.replace('[àäâ]','a',regex=True)\
.str.replace('[ïîì]','i',regex=True)\
.str.replace('[ôöù]','o',regex=True)\
.str.replace('[ûüù]','u',regex=True)\
.str.replace('^[a-z]{2}\\:','',regex=True)\
.str.replace('œ','oe',regex=True)

In [18]:
categories.describe()

,main_category,main_category_fr,traduit,main_category_fr_cor
count,3543,3543,1860,3543
unique,3543,3543,1660,3164
top,fr:filet-de-boeuf,Filet-de-boeuf,Produits laitiers,produits laitiers
freq,1,1,9,10


In [19]:
categories.loc[(categories.main_category_fr_cor.str.contains('s$').fillna(False)) & ~(categories.main_category_fr_cor.str.contains('frais$').fillna(False))].sample(5)

,main_category,main_category_fr,traduit,main_category_fr_cor
83576,en:frozen-donuts,en:Frozen-donuts,Beignets glacés,beignets glaces
22563,fr:coulis-de-framboises,Coulis-de-framboises,NaN,coulis de framboises
2,en:pastries,Pâtisseries,NaN,patisseries
37,en:chips-and-fries,Chips et frites,NaN,chips et frites
974,en:sweet-spreads,Produits à tartiner sucrés,NaN,produits a tartiner sucres


In [20]:
from Levenshtein import distance as levenshtein_distance
dico_proche = []

categories['main_category_fr_cor_2']=''

for ks in categories.loc[(categories.main_category_fr_cor.str.contains('s$').fillna(False)) & ~(categories.main_category_fr_cor.str.contains('frais$').fillna(False)),'main_category_fr_cor'].unique():
    ks_list = []
    
    for k in categories.main_category_fr_cor.unique():
        if ks is not np.nan and k  is not np.nan and ks != k and len(ks) > 4 and levenshtein_distance(ks,k) <= 2 and k[-1] != 's' and len(ks.split(' ')) >1:
            # les mots composés
            
            # add item in list
            dico_proche.append([ks,k])
            
        if ks is not np.nan and k  is not np.nan and ks != k and len(ks) > 4 and levenshtein_distance(ks,k) <= 1 and k[-1] != 's' and len(ks.split(' '))==1:
            # les mots simples
            
            # add item in list
            dico_proche.append([ks,k])
            
        #if k not in ['macaroni','păine','paine','toats','bigos','bison','alinos','aligot']:
            #categories.loc[categories.main_category_fr_cor == k,['main_category_fr_cor_2']] = ks

</br>

In [21]:
for ks in categories.loc[(categories.main_category_fr_cor.str.contains('s$').fillna(False)) & ~(categories.main_category_fr_cor.str.contains('frais$').fillna(False)),'main_category_fr_cor'].unique():
    for k in categories.main_category_fr_cor.unique():
        if ks is not np.nan and k  is not np.nan and ks != k and len(ks) > 4 and levenshtein_distance(ks,k) <= 2 and k[-1] != 's' and len(ks.split(' ')) >1:
            # les mots composés
            
            # add item in list
            dico_proche.append([ks,k])
            
        if ks is not np.nan and k  is not np.nan and ks != k and len(ks) > 4 and levenshtein_distance(ks,k) <= 1 and k[-1] != 's' and len(ks.split(' '))==1:
            # les mots simples
            
            # add item in list
            dico_proche.append([ks,k])
            
        #if k not in ['macaroni','păine','paine','toats','bigos','bison','alinos','aligot']:
            #categories.loc[categories.main_category_fr_cor == k,['main_category_fr_cor_2']] = ks

In [22]:
for ll in dico_proche:
    if k not in ['macaroni','păine','paine','toats','bigos','bison','alinos','aligot']:
        categories.loc[categories.main_category_fr_cor == ll[1],['main_category_fr_cor_2']] = ll[0]
    

In [23]:
categories[categories.main_category_fr_cor_2!=""].sample(10)

,main_category,main_category_fr,traduit,main_category_fr_cor,main_category_fr_cor_2
5140,fr:bruschetta,Bruschetta,NaN,bruschetta,bruschettas
10390,de:muffin,de:Muffin,Muffin,muffin,muffins
57677,en:sirop,en:Sirop,Sirop,sirop,sirops
1261,en:prepared-chicken,en:Prepared-chicken,Poulet préparé,poulet prepare,poulets prepares
65160,en:food-coulouring,en:Food-coulouring,Colorant alimentaire,colorant alimentaire,colorants alimentaires
50738,fr:puree-de-piment,Puree-de-piment,NaN,puree de piment,puree de piments
10586,es:pescado,es:Pescado,Poisson,poisson,poissons
74697,pt:biscoito-integral,pt:Biscoito-integral,Biscuit complet,biscuit complet,biscuits complets
65301,en:yeast,en:Yeast,Levure,levure,levures
65918,en:pate,en:Pate,Pate,pate,pates


</br>

In [24]:
for ks in categories.loc[(categories.main_category_fr_cor.str.contains('s$').fillna(False)) & ~(categories.main_category_fr_cor.str.contains('frais$').fillna(False)),'main_category_fr_cor'].unique():
    for k in categories.main_category_fr_cor.unique():
        
        if ks is not np.nan and k is not np.nan and ks != k and len(ks) > 4 and levenshtein_distance(ks,k) <= 2 and k[-1] != 's':

            # add item in list
            print(str(ks).ljust(25)+' | '+str(k).ljust(15) )
            
            if k not in ['macaroni','tomate','apero','macaroni','păine','mate','boeuf','paine','toats','bigos','pains de mie','bison','boisson','alinos','aligot']:
                categories.loc[categories.main_category_fr_cor == k,['main_category_fr_cor_2']] = ks

patisseries               | pattisserie    
patisseries               | patisserie     
pains                     | paine          
pains                     | păine          
volailles                 | volaille       
sirops                    | sirop          
sirops                    | siroop         
viandes                   | viande         
soupes                    | soupe          
charcuteries              | charcuterie    
chocolats                 | chocolat       
chocolats                 | chocolade      
chocolats                 | ciocolată      
boissons                  | poisson        
boissons                  | boisson        
sandwichs                 | sandwitch      
farines                   | farine         
farines                   | farina         
conserves                 | conserve       
yaourts                   | yaourt         
huiles                    | huile          
biscuits                  | biscuiți       
biscuits                  | bisc

In [25]:
categories[categories.main_category_fr_cor_2!=""].sample(10)

,main_category,main_category_fr,traduit,main_category_fr_cor,main_category_fr_cor_2
60275,en:biere,en:Biere,Biere,biere,bieres
81169,fr:preparation-pour-boisson,Preparation-pour-boisson,NaN,preparation pour boisson,preparations pour boissons
8204,pt:grao,pt:Grao,Grain,grain,graines
9985,en:gravy,en:Gravy,Sauce,sauce,sauces
83386,de:backware,de:Backware,Pâtisserie,patisserie,patisseries
73399,en:lait,en:Lait,Lait,lait,laits
1640,en:bruschetta,en:Bruschetta,Bruschetta,bruschetta,bruschettas
76250,en:biscotti,en:Biscotti,Biscotti,biscotti,biscottes
38732,fr:levain,Levain,NaN,levain,levains
76612,en:farina,en:Farina,Farina,farina,farines


In [26]:
categories.main_category_fr_cor_2[categories.main_category_fr_cor_2==""] = categories.main_category_fr_cor[categories.main_category_fr_cor_2==""]

</br>

In [27]:
categories.sample(10)

,main_category,main_category_fr,traduit,main_category_fr_cor,main_category_fr_cor_2
41592,fr:garnitures-pour-bouchees-a-la-reine,Garnitures-pour-bouchees-a-la-reine,NaN,garnitures pour bouchees a la reine,garnitures pour bouchees a la reine
63288,de:gemusemais,de:Gemusemais,Gemusemais,gemusemais,gemusemais
69357,en:biscuits-chocolates,en:Biscuits-chocolates,Biscuits chocolats,biscuits chocolats,biscuits chocolats
84241,en:fresh-raspberries,en:Fresh-raspberries,Framboises fraîches,framboises fraiches,framboises fraiches
3731,en:iced-green-tea,en:Iced-green-tea,Thé vert glacé,the vert glace,the vert glace
76317,it:yogurt-con-pezzi-di-frutta,it:Yogurt-con-pezzi-di-frutta,Yoghourt avec morceaux de fruits,yoghourt avec morceaux de fruits,yoghourt avec morceaux de fruits
9528,en:instant-mashed-potato,en:Instant-mashed-potato,Purée de pommes de terre instantanée,puree de pommes de terre instantanee,puree de pommes de terre instantanee
65187,de:gin,de:Gin,Gin,gin,gin
25966,fr:ananas-en-tranches,Ananas-en-tranches,NaN,ananas en tranches,ananas en tranches
1174,en:toppings,en:Toppings,Nappages,nappages,nappages


In [29]:
categories.describe()

,main_category,main_category_fr,traduit,main_category_fr_cor,main_category_fr_cor_2
count,3543,3543,1860,3543,3543
unique,3543,3543,1660,3164,3033
top,fr:filet-de-boeuf,Filet-de-boeuf,Produits laitiers,produits laitiers,chocolats
freq,1,1,9,10,12


In [30]:
categories = categories.rename({'main_category_fr_cor_2': 'correction_main_category_fr'}, axis=1)

In [31]:
categories.head()

,main_category,main_category_fr,traduit,main_category_fr_cor,correction_main_category_fr
0,fr:filet-de-boeuf,Filet-de-boeuf,NaN,filet de boeuf,filet de boeuf
1,en:fresh-vegetables,Légumes frais,NaN,legumes frais,legumes frais
2,en:pastries,Pâtisseries,NaN,patisseries,patisseries
3,en:breads,Pains,NaN,pains,pains
5,fr:boulange,Boulange,NaN,boulange,boulange


In [33]:
categories.loc[categories.correction_main_category_fr!='',['main_category_fr','correction_main_category_fr']]\
.to_csv("correction_main_category_fr.csv",sep='\t')